(c) 2025 Manuel Razo. This work is licensed under a [Creative Commons
Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).
All code contained herein is licensed under an [MIT
license](https://opensource.org/licenses/MIT).

In [ ]:
# Import base libraries
import os
import glob
import gc
import pickle
import warnings

# Import JAX-related libraries
import jax
from jax import random
import jax.numpy as jnp
from numpyro.optim import Adam
import numpyro

# Import numpy for array manipulation
import numpy as np
# Import statsmodels for statistical modeling
import statsmodels.api as sm
# Import scipy for special functions
import scipy as sp
# Import scikit-learn for PCA
from sklearn.decomposition import PCA
# Import umap-learn for UMAP
from umap import UMAP
# Import library to load h5ad files
import scanpy as sc
# Import pandas for data manipulation
import pandas as pd
# Import scribe
import scribe
# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
# Set plotting style
scribe.viz.matplotlib_style()

# Import colors
color_palette = scribe.viz.colors()

## Exploratory data analysis of `10xGenomics 50%:50% Jurkat:293T Cell Mixture` dataset

In this notebook, we will perform an exploratory data analysis of the `10xGenomics 50%:50% Jurkat:293T Cell Mixture` dataset. This is a public dataset provided by 10xGenomics with a mixture of Jurkat and 293T cells.

## Raw counts exploratory data analysis

Let's begin by listing the files in the dataset directory.

In [ ]:
# Define the path to the dataset
DATA_DIR = f"{scribe.utils.git_root()}/data/10xGenomics/50-50_Jurkat-293T_mixture"

# List all files in directory
files = glob.glob(f"{DATA_DIR}/*")

files

The data is contained in the `matrix.mtx` file. We can load it using `scanpy.read_mtx`.

In [ ]:
# Load the data
data = sc.read_mtx(f"{DATA_DIR}/matrix.mtx")

# Transpose the matrix to have cells as rows and genes as columns
data = data.T  

data

This file format is a bit unusual as the metadata is not contained in the file, but in separate files. Let's load the metadata files.

In [ ]:
# Load the genes.tsv file
genes = pd.read_csv(f"{DATA_DIR}/genes.tsv", sep="\t", header=None)

genes.head(5)

In [ ]:
# Load the barcodes.tsv file
barcodes = pd.read_csv(f"{DATA_DIR}/barcodes.tsv", sep="\t", header=None)

barcodes.head(5)

This is the metadata that should be included in the `matrix.mtx` file. Let's add it to the `data` object.

In [ ]:
# Transpose the data matrix and add metadata
data.var_names = genes.iloc[:, 0]  # Now genes (var) should match
data.obs_names = barcodes.iloc[:, 0]  # And barcodes (obs) should match

# Optional: Add gene symbols as additional metadata
data.var['name'] = genes.iloc[:, 1].values

data

Let's save this object as an `h5ad` file.

In [ ]:
# Save the data object to an h5ad file
data.write_h5ad(f"{DATA_DIR}/data.h5ad")

# Convert the data object to a dataframe
df = data.to_df()

df.head(5)

Now we can begin the exploratory data analysis.

Let's begin by plotting the distribution of total UMI counts per cell.

In [ ]:
# Initialize figure
fig, axes = plt.subplots(1, 2, figsize=(6, 3))

# Compute total counts per cell removing the first column (gene names)
total_counts = df.sum(axis=1)

# Plot the distribution of total counts per cell
sns.histplot(total_counts, ax=axes[0], color=color_palette['blue'])
sns.ecdfplot(total_counts, ax=axes[1], color=color_palette['blue'])

# Set title and labels
axes[0].set_xlabel('total UMI counts')
axes[0].set_ylabel('# cells')
axes[1].set_xlabel('total UMI counts')
axes[1].set_ylabel('ECDF')

plt.tight_layout()

This is a reasonably looking distribution. There are no extreme outliers on either end of the distribution. However, there is an obvious cutoff on the left side of the distribution, where cells with very few UMIs must have been removed.

Let's now perfomr PCA on the data and plot the first two principal components.

In [ ]:
# Perform PCA on the data
pca = PCA(n_components=2)
pca_result = pca.fit_transform(df.values)

# Initialize figure
fig, ax = plt.subplots(figsize=(4, 3))

# Plot the first two principal components
scatter = ax.scatter(
    pca_result[:, 0], 
    pca_result[:, 1], 
    c=total_counts, 
    cmap='viridis',
    s=8
)

# Label axes
ax.set_xlabel('Principal Component 1')
ax.set_ylabel('Principal Component 2')
# Set title
ax.set_title('PCA of 10xGenomics \n50%:50% Jurkat:293T Cell Mixture')

plt.colorbar(scatter, ax=ax, label='total UMI counts')

From this projection, we can sort of see a separation between the two cell types. However, it is not very clear. Let's try using UMAP.

In [ ]:
# Initialize UMAP
umap = UMAP(n_components=2)
# Fit UMAP
umap_result = umap.fit_transform(df.values)

Now, let's plot the UMAP.

In [ ]:
# Initialize figure
fig, ax = plt.subplots(figsize=(4, 3))

# Plot the UMAP
scatter = ax.scatter(
    umap_result[:, 0], 
    umap_result[:, 1], 
    c=total_counts, 
    cmap='viridis',
    s=8
)

# Add colorbar
plt.colorbar(scatter, ax=ax, label='total UMI counts')

# Label axes
ax.set_xlabel('UMAP 1')
ax.set_ylabel('UMAP 2')
# Set title
ax.set_title('UMAP of 10xGenomics \n50%:50% Jurkat:293T Cell Mixture')


UMAP **clearly** shows a separation between the two cell types divided by the first UMAP dimension. Let's quantify the proportion of cells on each side of the UMAP dimension.

In [ ]:
# Quantify proportion of cells with UMAP 1 < 5
count_low = np.mean(umap_result[:, 0] < 5)
count_high = np.mean(umap_result[:, 0] > 5)

# Print results
print(f"Proportion of cells with UMAP 1 < 5: {count_low / (count_low + count_high):.2%}")
print(f"Proportion of cells with UMAP 1 > 5: {count_high / (count_low + count_high):.2%}")